# **NLP à ressources limitées**

<img src="https://dli2024prac.blob.core.windows.net/content/lr_llm_header.png" width="60%" allign ="center"/>

<a href="https://colab.research.google.com/github/deep-learning-indaba/indaba-pracs-2024/blob/main/practicals/Indaba_2024_Prac_Template.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a> [Change colab link to point to prac.]

© Deep Learning Indaba 2024. Licence Apache 2.0.

**Auteurs :**
- Ali Zaidi
- Aya Salama
- Khalil Mrini
- Steven Kolawole 

**Introduction :**

Le TAL à ressources limitées (Traitement Automatique des Langues, ou NLP en anglais, Natural Language Processing) fait référence à l'étude et au développement de modèles et de systèmes TAL pour des langues, des tâches ou des domaines qui disposent de peu de données et de ressources. Cela peut inclure des langues avec moins de corpus textes numériques, d'outils informatiques limités ou de recherches linguistiques moins développées.

**Principaux défis du TAL à ressources limitées**

1. **Rareté des données :**
   - **Données d'entraînement limitées :** De nombreuses langues manquent de grands corpus annotés nécessaires pour entraîner les modèles TAL.
   - **Absence de modèles pré-entraînés :** Les modèles TAL populaires comme BERT, GPT et autres ne sont souvent pas disponibles pour les langues à ressources limitées.

2. **Diversité linguistique :**
   - **Complexité morphologique :** Certaines langues ont des structures grammaticales complexes et une richesse morphologique.
   - **Variations dialectales :** Un manque de versions standardisées peut compliquer les tâches TAL.

3. **Limitations des ressources :**
   - **Contraintes computationnelles :** Les scénarios à faibles ressources impliquent souvent un accès limité à la puissance de calcul et au stockage.
   - **Expertise et outils :** Moins d'experts linguistiques et moins d'outils TAL sont adaptés à ces langues.

**Sujets :**

Contenu : [Traitement Automatique des Langues, Ressources Limitées, Modèles de Langages, Réglage Fin Efficace en Paramètres, Adaptation]  
Niveau : [Intermédiaire]


**Objectifs/Compétences visées :**

- Explorer les défis de la rareté des données
- Explorer les limitations des ressources de calcul et les aborder avec un finetuning efficace des paramètres
- Comparer les performances entre les petits (BERT) et les grands (GPT) modèles de langage sur des langues/tâches à faibles ressources.
  

**Prérequis :**

[Connaissances requises pour cette pratique. Vous pouvez lier une session de track parallèle pertinente, des blogs, des articles, des cours, des sujets, etc.]
_ lier des ressources sur les LLMs, BERT, articles de Masakhane sur les conversations à faibles ressources

**Plan :**

[Points qui lient chaque section. Auto-générer en suivant les instructions [ici](https://stackoverflow.com/questions/67458990/how-to-automatically-generate-a-table-of-contents-in-colab-notebook).]

**Avant de commencer :**

[Tâches juste avant de commencer.]


# Configuration

## Exécutez la cellule pour configurer les packages Python et ressources nécessaires

**Dossiers de ressources** :

Les ressources dont vous aurez besoin pour réaliser ce pratique seront téléchargées lorsque vous exécuterez la prochaine cellule.
Une fois le téléchargement et l'extraction terminés, vous aurez les dossiers suivants présents dans le dossier "resources" dans le répertoire parent :

- *models* : ce dossier contient les modèles pré-entraînés qui seront utilisés dans le pratique
- *dataset* : ce dossier contient le jeu de données Goud-sum que nous utiliserons dans le pratique
- *generated_responses* : ce dossier contient des résumés pré-générés qui seront utilisés dans la Section 2

In [ ]:
import utils

# Installer les packages requis
utils.install_requirements()

# Téléchargez et extrayez le fichier zip contenant les ressources
utils.download_and_extract_zip("https://dli2024prac.blob.core.windows.net/resources/resources.zip")


## Imports

In [ ]:
import os
import time
import random

import numpy as np
import pandas as pd
import torch
from dotenv import load_dotenv
from tqdm import tqdm

from datasets import load_dataset, concatenate_datasets, load_metric, load_from_disk
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training, TaskType, PeftConfig, PeftModel
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainer, Seq2SeqTrainingArguments
from huggingface_hub import login
from rouge_metric import PyRouge
import wandb
import openai

In [ ]:
load_dotenv()
login(token=os.getenv("HF_HUB_TOKEN"))
wandb.login()

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /home/alizaidi/.cache/huggingface/token
Login successful


wandb: Currently logged in as: alizaidi. Use `wandb login --relogin` to force relogin


True

# Introduction

Dans ce pratique, nous nous intéressons à générer des titres pour les articles de presse présentés sur le site d'actualités [Goud.ma](www.Gound.ma).

Nous aborderons cela comme une tâche de résumé où l'entrée est le corps d'un article de presse et la sortie est un titre approprié. L'[ensemble de données Goud](https://github.com/issam9/goud-summarization-dataset) contient 158k articles et leurs titres. Tous les titres sont en Darija marocaine, tandis que les articles peuvent être en Darija marocaine, en arabe standard moderne, ou un mélange des deux (Darija marocaine en alternance de code).

**Champs de données**
- *article* : une chaîne de caractères contenant le corps de l'article de presse
- *headline* : une chaîne de caractères contenant le titre de l'article
- *categories* : une liste de chaînes de caractères des catégories d'articles

## Ce que nous allons faire :
<img src="./content/DLI_LR_llm_prac_1.png" width="40%" />

**Dossiers de ressources** :

Les ressources dont vous aurez besoin pour réaliser cet exercice font partie du dépôt, vous les trouverez dans le dossier parent :

- Dossier *Data* : ce dossier contient le dataset Goud-sum que nous utiliserons dans la pratique
- Dossier *genrated_responses* : ce dossier contient des résumés pré-générés qui seront utilisés dans la Section 2

## Métrique d'évaluation : ROUGE

ROUGE (Recall-Oriented Understudy for Gisting Evaluation) est un ensemble de métriques utilisé pour évaluer la qualité des résumés en les comparant aux résumés de référence (ou ground truth). ROUGE est largement utilisé dans les tâches de Traitement Automatique du Langage Naturel (NLP), en particulier pour évaluer la performance des modèles de résumé de texte.

![ROUGE-Base](https://i0.wp.com/blog.uptrain.ai/wp-content/uploads/2024/01/rouge-n.webp?resize=700%2C228&ssl=1)

### Variants clés de ROUGE

1. **ROUGE-N** : Mesure le chevauchement des n-grams entre le résumé candidat et le résumé de référence.

![ROUGE-1](https://clementbm.github.io/assets/2021-12-23/rouge-unigrams.png)

*caption:*
$ROUGE_1 = \frac{7}{10} = 0.7$

   - **ROUGE-1** : Chevauchement des unigrams (1-gram).
   - **ROUGE-2** : Chevauchement des bigrams (2-grams).
   - **ROUGE-L** : Mesure la plus longue sous-séquence commune (LCS) entre les résumés candidat et de référence.

2. **ROUGE-L** : Mesure la plus longue sous-séquence commune (LCS) entre le résumé candidat et le résumé de référence. Contrairement à ROUGE-N, ROUGE-L considère la similarité de la structure au niveau de la phrase en identifiant la plus longue séquence de mots co-occurrences dans les deux résumés.

3. **ROUGE-W** : Une version pondérée de ROUGE-L qui donne plus d'importance à la LCS contiguë.

4. **ROUGE-S** : Mesure le chevauchement des skip-bigrams, qui sont des paires de mots dans leur ordre d'apparition pouvant avoir n'importe quel nombre d'écarts entre eux.

### Comment ROUGE est calculé

Les métriques ROUGE peuvent être calculées en termes de trois mesures :

- **Recall** : Le ratio des unités chevauchantes (n-grams, LCS, ou skip-bigrams) entre le résumé candidat et le résumé de référence par rapport au nombre total d'unités dans le résumé de référence. Cela répond à la question : "Quelle part du résumé de référence est capturée par le résumé candidat ?".

- **Précision** : Le ratio des unités chevauchantes entre le résumé candidat et le résumé de référence par rapport au nombre total d'unités dans le résumé candidat. Cela répond à la question : "Quelle part du résumé candidat est pertinente par rapport au résumé de référence ?".

- **F1-Score** : La moyenne harmonique de Précision et Recall. Cela donne une mesure équilibrée qui considère à la fois la précision et le recall.

### Importance de ROUGE

ROUGE est essentiel pour les tâches de résumé car il fournit un moyen normalisé pour évaluer et comparer différents modèles de résumé. Des scores ROUGE plus élevés indiquent généralement que le résumé candidat est plus similaire au résumé de référence, ce qui signifie que le modèle fonctionne probablement bien.

#### NOTE: Mise en garde
<div style="display: flex; justify-content: space-between;">
    <img src="https://miro.medium.com/v2/resize:fit:720/format:webp/1*8ZNpaag-Nr2GLs3A-sz0aQ.png" alt="limitation 1" width="250"/>
    <img src="https://miro.medium.com/v2/resize:fit:720/format:webp/1*CLIKeyKYiR6sNA4yjIkCWg.png" alt="limitation 2" width="250"/>
    <img src="https://miro.medium.com/v2/resize:fit:720/format:webp/1*667HMbjSLJhwR_xqBau3JQ.png" alt="limitation 3" width="250"/>
</div>

Bien que ROUGE et d'autres métriques d'évaluation (par exemple, BLEU, METEOR, etc.) servent d'outils précieux pour une évaluation rapide et simple des modèles de langue, elles présentent certaines limitations qui les rendent moins idéales. Pour commencer, elles ne parviennent pas à évaluer la fluidité, la cohérence et le sens général des passages. Elles sont également relativement insensibles à l'ordre des mots. ROUGE mesure principalement le chevauchement lexical et peut ne pas capturer pleinement le sens sémantique ou la qualité d'un résumé. Pour ces raisons, les chercheurs cherchent encore à trouver des métriques améliorées.

Par conséquent, ces métriques ne remplacent pas totalement l'évaluation humaine, mais elles sont mieux utilisées conjointement avec les évaluations humaines pour une évaluation plus complète de la qualité des résumés.

## Charger le dataset

In [5]:
from datasets import load_from_disk
# Load the dataset from disk
dataset = load_from_disk("./data/Goud-sum/Goud-sum")

## Vérifier le dataset

In [ ]:
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['article', 'headline', 'categories'],
        num_rows: 139288
    })
    validation: Dataset({
        features: ['article', 'headline', 'categories'],
        num_rows: 9497
    })
    test: Dataset({
        features: ['article', 'headline', 'categories'],
        num_rows: 9497
    })
})


In [ ]:
dataset['train'][0]

{'article': 'منير العلمي من مراكش: تحول فضاء مقر الغرفة الفلاحية بمدينة مراكش، الذي يحتضن في هذه الأثناء، انتخاب رئيس وأعضاء المكتب المسير للغرفة الفلاحية بجهة مراكش آسفي، إلى حلبة للاشتباكات والملاسنات، بعد اشتداد الخلاف بين البرلمانيين حميد العكرود وعمر خفيف، اللذين ينتميان إلى حزب التجمع الوطني للأحرار، ما كاد يعصف بالاجتماع بعد انطلاق شرارة الاشتباك بالأيادي التي أجهضت في مهدها بتدخل بعض الحاضرين. وحسب شهود عيان، فإن عمر خفيف، الذي يشغل رئيس جماعة أكفاي، ومدعم الحبيب بن الطالب المنسق الاقليمي لحزب الأصالة والمعاصر الذي يتجه لتولي رئاسة الغرفة لولاية تانية، رفض دخول حميد العكرود للمنافسة على رئاسة الغرفة، واصفا إياه بـ “الأمي الذي لايفقه شيئا”، ليدخل الطرفان في ملاسنات كلامية قبل أن يتحول الصراع إلى تشابك بالأيدي. ',
 'headline': 'برلمانيين من حزب الحمامة قلبوها بونيا قبل انتخاب رئيس وأعضاء غرفة الفلاحة بجهة مراكش آسفي (صور)',
 'categories': "['آش واقع', 'الرئيسية']"}

# Section1: Affiner efficacement les modèles Seq2Seq avec Low Rank Adaptation (LoRA)

Nous allons utiliser Hugging Face [Transformers](https://huggingface.co/docs/transformers/index), [Accelerate](https://huggingface.co/docs/accelerate/index), et [PEFT](https://github.com/huggingface/peft). 

Vous apprendrez à :

1. Configurer l'environnement de développement
2. Charger et préparer le jeu de données
3. Affiner Multilingual BERT avec LoRA et bnb int-8
4. Évaluer et exécuter une inférence
5. Comparaison des performances de coût

### Introduction rapide à PEFT ou Affinage de paramètres efficace

<div style="display: flex; justify-content: center; align-items: flex-start;">    <figure style="text-align: center;">
        <a href="https://arxiv.org/abs/2303.15647#" target="_blank">
            <img src="./content/PEFT_method.png" width="90%" />
        </a>
        <figcaption><a href="https://arxiv.org/abs/2303.15647#" target="_blank">Méthodes PEFT, de l'article "Scaling Down to Scale Up: A Guide to Parameter-Efficient Fine-Tuning"
</a></figcaption>
    </figure>
</div>

[PEFT](https://github.com/huggingface/peft), ou Affinage de paramètres efficace, est une nouvelle bibliothèque open-source de Hugging Face permettant l'adaptation efficace des modèles de langue pré-entrainés (PLMs) à diverses applications en aval sans affiner tous les paramètres du modèle. PEFT inclut actuellement des techniques pour :

- LoRA: [LORA: LOW-RANK ADAPTATION OF LARGE LANGUAGE MODELS](https://arxiv.org/pdf/2106.09685.pdf)
- Prefix Tuning: [P-Tuning v2: Prompt Tuning Can Be Comparable to Fine-tuning Universally Across Scales and Tasks](https://arxiv.org/pdf/2110.07602.pdf)
- P-Tuning: [GPT Understands, Too](https://arxiv.org/pdf/2103.10385.pdf)
- Prompt Tuning: [The Power of Scale for Parameter-Efficient Prompt Tuning](https://arxiv.org/pdf/2104.08691.pdf)

## Adaptation à Basse Rangée (LoRA)

Bien que les grands modèles de langage (LLMs) aient montré des performances remarquables dans une large gamme de tâches de NLP, ils nécessitent des ressources de calcul importantes pour l'entraînement, le fine-tuning et le déploiement. De plus, de nombreux cas d'utilisation du monde réel nécessitent l'adaptation des LLMs disponibles à leur tâche cible afin d'atteindre les performances souhaitées.

Alors que le fine-tuning d'un LLM complet est prohibitivement coûteux, même sur de petits ensembles de données. Par exemple, le fine-tuning complet du modèle Llama7B nécessite 112 Go de VRAM, soit au moins deux GPU A100 de 80 Go. Heureusement, des méthodes de fine-tuning efficaces en termes de paramètres comme LoRA permettent aux utilisateurs avec des ressources limitées d'adapter efficacement et efficacement un LLM à leur tâche cible.

Dans ce tutoriel, nous explorons QLoRA, qui est une technique de fine-tuning efficace en termes de paramètres qui réduit le nombre de paramètres ajustés pendant le processus d'adaptation, et introduit en plus une quantification pour réduire encore l'empreinte mémoire du modèle adapté.

### Comment Fonctionne LoRA ?

L'article [LoRA: Low-Rank Adaptation of Large Language Models](https://arxiv.org/abs/2106.09685) s'inspire de la conjecture selon laquelle les modèles surparamétrés couvrent une dimension intrinsèque à basse rangée. Une basse dimension intrinsèque signifie que les données peuvent être efficacement représentées ou approchées par un espace de dimension inférieure tout en conservant la plupart de leurs informations ou structures essentielles. En d'autres termes, cela signifie que nous pouvons décomposer la nouvelle matrice de poids pour la tâche adaptée en matrices de dimension inférieure (plus petites) sans perdre d'informations significatives.

Concrètement, supposons que $\delta W$ soit la mise à jour des poids pour une matrice de poids de $A\times B$. Alors, une décomposition à basse rangée de $\delta W$ peut être exprimée comme : $\delta W = W_A W_B$, où $W_A$ est une matrice de $A\times k$ et $W_B$ est une matrice de $k\times B$. Ici, $k$ est le rang de la décomposition, et est généralement beaucoup plus petit que $A$ et $B$.

![Image courtoisie du tutoriel de Sebastian Raschka sur LoRA de Lightning.AI](https://lightningaidev.wpengine.com/wp-content/uploads/2023/04/lora-4-300x226@2x.png)

## Résumé en utilisant mT5

Avant l'affinage de notre modèle, nous devons sélectionner le modèle que nous utiliserons comme modèle de base. Dans ce cas, nous utiliserons le modèle [mT5](https://huggingface.co/google/mt5-small), qui est une variante multilingue du modèle T5. Le modèle mT5 est entraîné sur un large corpus multilingue et est capable de réaliser une vaste gamme de tâches en PNL, y compris le résumé.

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("google/mt5-small")
model = AutoModelForSeq2SeqLM.from_pretrained("google/mt5-small")

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


/home/alizaidi/micromamba/envs/peft/lib/python3.10/site-packages/transformers/convert_slow_tokenizer.py:562: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


Ensuite, nous préparons nos jeux de données pour l'entraînement. Cela nécessite de tokenizer les séquences d'entrée et de sortie, de les compléter jusqu'à la longueur souhaitée, puis de les convertir en objets Dataset de PyTorch.

In [ ]:
# The maximum total input sequence length after tokenization.
# Sequences longer than this will be truncated, sequences shorter will be padded.
tokenized_inputs = concatenate_datasets([dataset["train"], dataset["test"]]).map(
    lambda x: tokenizer(x["article"], truncation=True),
    batched=True,
    remove_columns=["categories", "headline"],
)
input_lenghts = [len(x) for x in tokenized_inputs["input_ids"]]

In [ ]:
# take 85 percentile of max length for better utilization
max_source_length = int(np.percentile(input_lenghts, 85))
print(f"Max source length: {max_source_length}")

Max source length: 571


In [ ]:
# The maximum total sequence length for target text after tokenization.
# Sequences longer than this will be truncated, sequences shorter will be padded."
tokenized_targets = concatenate_datasets([dataset["train"], dataset["test"]]).map(
    lambda x: tokenizer(x["headline"], truncation=True),
    batched=True,
    remove_columns=["article", "categories"],
)
target_lenghts = [len(x) for x in tokenized_targets["input_ids"]]
# take 90 percentile of max length for better utilization
max_target_length = int(np.percentile(target_lenghts, 90))
print(f"Max target length: {max_target_length}")

Max target length: 50


In [ ]:
def preprocess_function(sample, padding="max_length"):
    # # add prefix to the input for t5
    # inputs = ["summarize: " + item for item in sample["dialogue"]]

    # tokenize inputs
    model_inputs = tokenizer(sample['article'], max_length=max_source_length, padding=padding, truncation=True)

    # Tokenize targets with the `summary` keyword argument
    labels = tokenizer(sample["headline"], max_length=max_target_length, padding=padding, truncation=True)

    # If we are padding here, replace all tokenizer.pad_token_id in the labels by -100 when we want to ignore
    # padding in the loss.
    if padding == "max_length":
        labels["input_ids"] = [
            [(l if l != tokenizer.pad_token_id else -100) for l in label] for label in labels["input_ids"]
        ]

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_dataset = dataset.map(preprocess_function, batched=True, remove_columns=["headline", "article", "categories"])
print(f"Keys of tokenized dataset: {list(tokenized_dataset['train'].features)}")

Keys of tokenized dataset: ['input_ids', 'attention_mask', 'labels']


In [ ]:
tokenized_dataset["test"].save_to_disk("arabic-goud-data/eval")

Saving the dataset (0/1 shards):   0%|          | 0/9497 [00:00<?, ? examples/s]

Enfin, nous devons définir notre configuration pour LoRA. Les principaux paramètres pour LoRA sont :

* `r` : il s'agit du rang des matrices décomposées $A$ et $B$ à apprendre pendant le fine-tuning. Un nombre plus petit économisera plus de mémoire GPU mais pourrait diminuer les performances.
* `lora_alpha` : il s'agit du poids de la perte de bas-rang dans la fonction de perte totale, ou du coefficient pour le facteur $\Delta W$ appris. Un nombre plus grand entraînera généralement un changement de comportement plus significatif après le fine-tuning.
* `lora_dropout` : le ratio de dropout pour les couches dans les adaptateurs LoRA $A$ et $B$.
* `target_modules` : les modules pour lesquels apprendre la décomposition de bas-rang. Cela pourrait être toutes les couches linéaires, par exemple, ou des modules spécifiques du réseau de base.

In [ ]:
lora_config = LoraConfig(
  r=16,
  lora_alpha=32,
  target_modules=["q", "v"],
  lora_dropout=0.05,
  bias="none",
  task_type=TaskType.SEQ_2_SEQ_LM
)
# prepare int-8 model for training
model = prepare_model_for_kbit_training(model)

# add LoRA adaptor
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

trainable params: 688,128 || all params: 300,864,896 || trainable%: 0.2287


In [ ]:
# we want to ignore tokenizer pad token in the loss
label_pad_token_id = -100
# Data collator
data_collator = DataCollatorForSeq2Seq(
    tokenizer=tokenizer,
    model=model,
    label_pad_token_id=label_pad_token_id,
    pad_to_multiple_of=8,
    return_tensors='pt'
)

In [ ]:
NUM_EPOCHS = 10
output_dir = "lora-goud-mt5-small"

In [ ]:
# Define training args
training_args = Seq2SeqTrainingArguments(
    output_dir="lora-mt5-goud",
    auto_find_batch_size=True,
    learning_rate=1e-3,
    num_train_epochs=NUM_EPOCHS,
    logging_dir=f"{output_dir}/logs",
    logging_strategy="steps",
    logging_steps=500,
    save_strategy="no",
    report_to=["tensorboard", "wandb"],
)

# Create Trainer instance
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset = tokenized_dataset["train"],
    eval_dataset = tokenized_dataset["validation"].select(range(20)),
    tokenizer = tokenizer,
)

In [ ]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


wandb: wandb version 0.17.7 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


wandb: Tracking run with wandb version 0.17.6


wandb: Run data is saved locally in /home/alizaidi/dev/nlp/llms/indaba/indaba-low-resource-nlp-prac/wandb/run-20240816_012514-brfaca9p
wandb: Run `wandb offline` to turn off syncing.


wandb: Syncing run lora-mt5-goud


wandb: ⭐️ View project at https://wandb.ai/alizaidi/huggingface


wandb: 🚀 View run at https://wandb.ai/alizaidi/huggingface/runs/brfaca9p


Step,Training Loss
500,6.364100
1000,5.019700
1500,4.733600
2000,4.573900
2500,4.487000
3000,4.441000
3500,4.375900
4000,4.370600
4500,4.312700
5000,4.309900


TrainOutput(global_step=174110, training_loss=3.7825945418129274, metrics={'train_runtime': 17642.8655, 'train_samples_per_second': 78.949, 'train_steps_per_second': 9.869, 'total_flos': 8.31857984054231e+17, 'train_loss': 3.7825945418129274, 'epoch': 10.0})

In [ ]:
peft_model_id="peft-lora-mt5-goud-results"

In [ ]:
trainer.model.save_pretrained(peft_model_id)
tokenizer.save_pretrained(peft_model_id)

('peft-lora-mt5-goud-results/tokenizer_config.json',
 'peft-lora-mt5-goud-results/special_tokens_map.json',
 'peft-lora-mt5-goud-results/spiece.model',
 'peft-lora-mt5-goud-results/added_tokens.json',
 'peft-lora-mt5-goud-results/tokenizer.json')

In [ ]:
trainer.push_to_hub("alizaidi/lora-mt5-goud-ar")

tokenizer.json:   0%|          | 0.00/16.3M [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/2.77M [00:00<?, ?B/s]

Upload 4 LFS files:   0%|          | 0/4 [00:00<?, ?it/s]

training_args.bin:   0%|          | 0.00/5.30k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/4.31M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/alizaidi/lora-mt5-goud/commit/fee52adbeda959a97d9b839e0b8f0a5f315b0713', commit_message='alizaidi/lora-mt5-goud-ar', commit_description='', oid='fee52adbeda959a97d9b839e0b8f0a5f315b0713', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
config = PeftConfig.from_pretrained(peft_model_id)

In [ ]:
# load base LLM model and tokenizer
model = AutoModelForSeq2SeqLM.from_pretrained(config.base_model_name_or_path,  load_in_8bit=True,  device_map={"":0})
tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


/home/alizaidi/micromamba/envs/peft/lib/python3.10/site-packages/transformers/convert_slow_tokenizer.py:562: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [ ]:
# Load the Lora model
model = PeftModel.from_pretrained(model, peft_model_id, device_map={"":0})

text = dataset["test"][0]["article"]
inputs = tokenizer(text, return_tensors="pt")

with torch.no_grad():
    outputs = model.generate(input_ids=inputs["input_ids"].to("cuda"), max_new_tokens=128)
    print(tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True)[0])

بالفيديو. طالبة كليات فاس خرجات للاحتجاج على فتح الأحياء والجامعات


In [ ]:
dataset["test"][0]["headline"]

'روبورطاج.. البياتة بالليل على برا شكل احتجاجي جديد للمطالبة بفتح الأحياء الجامعية بفاس'

In [ ]:
def generate_batch_sized_chunks(list_of_elements, batch_size):
    """
    split the dataset into smaller batches that we can process simultaneously
    Yield successive batch-sized chunks from list_of_elements.
    """
    for i in range(0, len(list_of_elements), batch_size):
        yield list_of_elements[i : i + batch_size]


def calculate_metric_on_test_ds(
    dataset,
    metric,
    model,
    tokenizer,
    batch_size=16,
    device="cuda" if torch.cuda.is_available() else "cpu",
    column_text="article",
    column_summary="highlights",
):
    article_batches = list(
        generate_batch_sized_chunks(dataset[column_text], batch_size)
    )
    target_batches = list(
        generate_batch_sized_chunks(dataset[column_summary], batch_size)
    )

    for article_batch, target_batch in tqdm(
        zip(article_batches, target_batches), total=len(article_batches)
    ):

        inputs = tokenizer(
            article_batch,
            max_length=1024,
            truncation=True,
            padding="max_length",
            return_tensors="pt",
        )

        summaries = model.generate(
            input_ids=inputs["input_ids"].to(device),
            attention_mask=inputs["attention_mask"].to(device),
            length_penalty=0.8,
            num_beams=8,
            max_length=128,
        )
        """ parameter for length penalty ensures that the model does not generate sequences that are too long. """

        # Finally, we decode the generated texts,
        # replace the  token, and add the decoded texts with the references to the metric.
        decoded_summaries = [
            tokenizer.decode(
                s, skip_special_tokens=True, clean_up_tokenization_spaces=True
            )
            for s in summaries
        ]

        decoded_summaries = [d.replace("", " ") for d in decoded_summaries]
        metric.add_batch(predictions=decoded_summaries, references=target_batch)

    #  Finally compute and return the ROUGE scores.
    score = metric.compute()
    return score


def evaluation(tokenizer, model, dataset):
    device = "cuda" if torch.cuda.is_available() else "cpu"

    # loading data
    rouge_names = ["rouge1", "rouge2", "rougeL", "rougeLsum"]
    rouge_metric = load_metric("rouge")
    score = calculate_metric_on_test_ds(
        dataset["test"][0:10],
        rouge_metric,
        model,
        tokenizer,
        batch_size=2,
        column_text="article",
        column_summary="headline",
    )

    rouge_dict = dict((rn, score[rn].mid.fmeasure) for rn in rouge_names)

    return rouge_dict

In [ ]:
evaluation(tokenizer, model, dataset)

/tmp/ipykernel_498243/3742215907.py:70: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  rouge_metric = load_metric("rouge")



  0%|                                                                    | 0/5 [00:00<?, ?it/s]


 20%|████████████                                                | 1/5 [00:02<00:08,  2.03s/it]


 40%|████████████████████████                                    | 2/5 [00:04<00:06,  2.10s/it]


 60%|████████████████████████████████████                        | 3/5 [00:06<00:04,  2.23s/it]


 80%|████████████████████████████████████████████████            | 4/5 [00:10<00:02,  2.80s/it]


100%|████████████████████████████████████████████████████████████| 5/5 [00:13<00:00,  2.80s/it]


100%|████████████████████████████████████████████████████████████| 5/5 [00:13<00:00,  2.61s/it]

{'rouge1': 0.1, 'rouge2': 0.0, 'rougeL': 0.1, 'rougeLsum': 0.1}

## Évaluation1 : point de contrôle précoce du modèle

In [ ]:
#load an early checkpoint
#run evaluation 

## Évaluation2 : modèle final entraîné AraBERT, DarijaBERT

# Activité : Résumé d'Article en Langue Maternelle

**Tâche :** Rechercher un article dans votre langue maternelle et évaluer les capacités de résumé et de génération de titres de ChatGPT.

**Étapes :**

1. **Sélectionner un Article :** Choisissez un article pertinent et récent rédigé dans votre langue maternelle. Assurez-vous qu'il soit de longueur courte ou moyenne.

2. **Résumer avec ChatGPT :** Utilisez ChatGPT pour générer un résumé de l'article sélectionné.

3. **Évaluer la Qualité du Résumé :**
    - **Impression :** Partagez votre impression sur la qualité du résumé. Considérez les points suivants :
        - **Exactitude :** Le résumé capture-t-il les points principaux et l'essence de l'article ?
        - **Clarté :** Le résumé est-il clair et facile à comprendre ?
        - **Couverture :** Le résumé inclut-il toutes les informations cruciales de l'article ?

4. **Fournir un Retour :** Offrez un retour constructif sur le résumé. Soulignez les divergences éventuelles ou les domaines à améliorer.

In [18]:
# @title Generate Quiz Form. (Run Cell)
from IPython.display import HTML

HTML(
    """
<iframe
	src="https://forms.gle/sggLJWMFQ4JQCmHL8",
  width="80%"
	height="320px" >
	Loading...
</iframe>
"""
)

# Section2: Résumé en utilisant GPT


Dans cette section, nous allons utiliser à la fois les grands et petits modèles de langue d'OpenAI pour accomplir la même tâche de résumé textuel abstractive. Nous évaluerons ensuite leurs performances et comparerons les résultats avec ceux obtenus à partir des modèles discutés dans la Section 1.

Contrairement aux approches traditionnelles de fine-tuning qui impliquent la mise à jour des poids du modèle, la première étape de l'adaptation d'un modèle basé sur GPT pour une tâche spécifique est l'ingénierie des prompts, qui ne nécessite pas de mise à jour des poids.

<div style="display: flex; align-items: flex-start;">
    <figure style="margin-right: 10px; text-align: center;">
        <a href="https://arxiv.org/pdf/2005.14165" target="_blank">
            <img src="./content/traditional-finetuning.png" width="80%" />
        </a>
        <figcaption><a href="https://arxiv.org/pdf/2005.14165" target="_blank">Traditional Fine-Tuning</a></figcaption>
    </figure>
    <figure style= "text-align: center;">
        <a href="https://arxiv.org/pdf/2005.14165" target="_blank">
            <img src="./content/prompting.png" width="80%" />
        </a>
        <figcaption><a href="https://arxiv.org/pdf/2005.14165" target="_blank">Prompting</a></figcaption>
    </figure>
</div>


## Imports

In [2]:
import pandas as pd
from rouge_metric import PyRouge
from tqdm import tqdm
import os
import random
import openai
import time
from openai import OpenAI
import os 
import time

## Fonctions Utilitaires

In [3]:
def evaluate_rouge(hypotheses, references):
    these_refs = [[ref.strip().lower()] for ref in references]
    rouge = PyRouge(rouge_n=(1, 2), rouge_l=True)
    scores = rouge.evaluate(hypotheses, these_refs)
    print(scores)

def substring_after_colon(input_string):
    colon_index = input_string.find(':')
    if colon_index != -1:
        return input_string[colon_index + 1:]
    else:
        return input_string

In [6]:
# Define dataset and paths
DATASET = "Goud"
MAX_TRAIN = 0
model_name = "gpt-4o-mini"

output_filename = f"./{DATASET}_{model_name}_test_generated_{MAX_TRAIN}.csv"

# Load dataset
goud_data = dataset
train_source = goud_data["train"]["article"]
train_target = goud_data["train"]["headline"]
test_source = goud_data["test"]["article"]

## Fonction pour résumer des articles de presse

In [ ]:
key = ""

In [ ]:
def summarize_news_article(MAX_TRAIN=20):
       
    client = OpenAI(api_key=key)
    rewritten_prompt_count = 0
    line_count = 0
    wait_time = 1
    df_lines = []
    tokens_consumption = 0
    existing_len = 0
    if os.path.exists(output_filename):
        existing_df = pd.read_csv(output_filename)
        existing_len = existing_df.shape[0]
        rewritten_prompt_count = existing_len
        line_count = existing_len
        df_lines = existing_df.to_dict('records')
    
    for data in tqdm(test_source[existing_len:], desc=f"Lines processed from {existing_len}-th line"):
        news_article = data.strip()
        line_count += 1
        made_error = True
        num_error = 0
        while made_error:
            messages = [{"role": "system", "content": "You are asked to summarize a news article written in Modern Standard Arabic and Moroccan Darija, and write that summary as a clickbait headline, in Moroccan Darija only.\n"}]
            if MAX_TRAIN - num_error > 0:
                for _ in range(MAX_TRAIN - num_error):
                    idx = random.choice(range(len(train_source)))
                    train_src = train_source[idx]
                    train_tgt = train_target[idx]
                    messages.append({"role": "user", "content": f"Summarize the following news article into a headline in Moroccan Darija only:\n\"{train_src}\""})
                    messages.append({"role": "assistant", "content": f"Absolutely! Here is the headline summarizing your news article:\n\"{train_tgt}\""})
            messages.append({"role": "user", "content": f"Summarize the following news article into a headline in Moroccan Darija only:\n\"{news_article}\""})
            try:
                response = client.chat.completions.create(
                    messages=messages,
                    model=model_name,
                )
                headline = response.choices[0].message.content
                df_lines.append({"article": news_article,"generated_headline": headline,"prompt_messages":messages})

                rewritten_prompt_count += 1
                made_error = False
            except Exception as e:
                if isinstance(e, openai.RateLimitError):
                    print("Rate limit error")
                    print(f"Wait for {wait_time} seconds because all calls failed: ", flush=True)
                    time.sleep(wait_time)
                    wait_time *= 2
                else:
                    print(e)
                    num_error += 1
                    print("May be too long, reducing context to:", MAX_TRAIN - num_error)
            #time.sleep(1)
    
    df = pd.DataFrame.from_dict(df_lines)
    df.to_csv(output_filename)

## Exécuter la synthèse

In [ ]:
#record cell running time
import time
start_time = time.time()
summarize_news_article(0)
print("--- %s seconds ---" % (time.time() - start_time))

Lines processed from 0-th line: 100%|██████████| 9497/9497 [2:34:22<00:00,  1.03it/s]  


--- 9263.82370686531 seconds ---


## Charger le résultat généré et évaluer le ROUGE


Dans le dossier "generated_responses", vous trouverez les réponses gpt correspondant aux invites de 0, 1, 5, 20 coups.

Évaluez les résumés de titres générés en exécutant l'évaluation ROUGE et ajoutez les résultats au tableau des résultats.

### Résultats du Métrique ROUGE : 0 Shot

| Métrique | Rappel (r)         | Précision (p)      | Score F1 (f)      |
|----------|-------------------|-------------------|-------------------|
| ROUGE-1  | 0.1228            | 0.1069            | 0.1143            |
| ROUGE-2  | 0.0282            | 0.0235            | 0.0256            |
| ROUGE-L  | 0.1128            | 0.0980            | 0.1049            |

In [7]:
shot_count = 0
hypotheses = pd.read_csv(f".\generated_responses\{model_name}\Goud_{model_name}_test_generated_{str(shot_count)}.csv", encoding = "UTF-8")["generated_headline"].tolist()
hypotheses = [substring_after_colon(hypo).replace("\"", "").strip() for hypo in hypotheses]
references = goud_data["test"]["headline"]
evaluate_rouge(hypotheses, references)


{'rouge-1': {'r': 0.11884854492859614, 'p': 0.12977184865821972, 'f': 0.12407023545586798}, 'rouge-2': {'r': 0.0325784137233658, 'p': 0.03389945881811894, 'f': 0.03322581039836068}, 'rouge-l': {'r': 0.11128523451125509, 'p': 0.12142149826298465, 'f': 0.11613260817866634}}


### Résultats du Metric ROUGE: 20 Exemple
Choisissez un ou plusieurs des fichiers contenant les réponses GPT N-shot précédemment générées, présents dans le dossier "generated_responses", exécutez l'évaluation, puis remplissez le tableau ci-dessous

| Metric   | Rappel (r)        | Précision (p)     | Score-F (f)       |
|----------|-------------------|-------------------|-------------------|
| ROUGE-1  |                   |                   |                   |
| ROUGE-2  |                   |                   |                   |
| ROUGE-L  |                   |                   |                   |

In [16]:
shot_count = 20
model_name  = "gpt4"  #"C:\Users\salamaaya\OneDrive - Microsoft\Desktop\DLI\Indaba2024-practical\indaba-low-resource-nlp-prac\generated_responses\gpt4\Goud_test_generated_5.csv"
hypotheses = pd.read_csv(f".\generated_responses\{model_name}\Goud_test_generated_{str(shot_count)}.csv", encoding = "UTF-8")["generated_headline"].tolist()
hypotheses = [substring_after_colon(hypo).replace("\"", "").strip() for hypo in hypotheses]
references = goud_data["test"]["headline"]
evaluate_rouge(hypotheses, references)

{'rouge-1': {'r': 0.13750397869020445, 'p': 0.1306205269799418, 'f': 0.13397389480280544}, 'rouge-2': {'r': 0.03387859852289136, 'p': 0.03194819653833152, 'f': 0.032885092553751126}, 'rouge-l': {'r': 0.1264128630910928, 'p': 0.11986076103165903, 'f': 0.12304965282629712}}


In [ ]:
# @title Generate Quiz Form. (Run Cell)
from IPython.display import HTML

HTML(
    """
<iframe
	src="https://forms.gle/zbJoTSz3nfYq1VrY6",
  width="80%"
	height="1200px" >
	Loading...
</iframe>
"""
)

## Conclusion
**Résumé :**

[Résumé des points principaux/à retenir de la pratique.]

**Prochaines Étapes :**

[Prochaines étapes pour les personnes ayant terminé la pratique, comme des lectures optionnelles (par exemple, blogs, articles, cours, vidéos YouTube). Cela pourrait également renvoyer à d'autres pratiques.]

**Annexe :**

[Tout ce qui (probablement des trucs mathématiques lourds) ne trouve pas de place dans les sections pratiques principales.]

**Références :**

[Références pour tout contenu utilisé dans le notebook.]

Pour d'autres pratiques du Deep Learning Indaba, veuillez visiter [ici](https://github.com/deep-learning-indaba/indaba-pracs-2022).

## Donnez-nous votre avis sur notre session!

Veuillez fournir des retours que nous pouvons utiliser pour améliorer nos travaux pratiques à l'avenir.

In [ ]:
# @title Generate Feedback Form. (Run Cell)
from IPython.display import HTML

HTML(
    """
<iframe
	src="https://forms.gle/WUpRupqfhFtbLXtN6",
  width="80%"
	height="1200px" >
	Loading...
</iframe>
"""
)

<img src="https://baobab.deeplearningindaba.com/static/media/indaba-logo-dark.d5a6196d.png" width="50%" />